# 输出显卡

In [4]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU.")
print(f"Available GPUs: {torch.cuda.device_count()}")
print(f"Current GPU: {torch.cuda.current_device()}")

CUDA available: True
PyTorch version: 2.5.1
GPU: NVIDIA GeForce RTX 2060
Available GPUs: 1
Current GPU: 0


# 输出模型结构

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device_id = 0  # ← 修改这里选择GPU编号
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")

model_path = r"F:\edged\cpm"

# 加载模型时指定设备
model = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)  # 将模型直接加载到指定设备
print(model)

MiniCPM3ForCausalLM(
  (model): MiniCPM3Model(
    (embed_tokens): Embedding(73448, 2560)
    (layers): ModuleList(
      (0-61): 62 x MiniCPMDecoderLayer(
        (self_attn): MiniCPMSdpaAttention(
          (q_a_proj): Linear(in_features=2560, out_features=768, bias=False)
          (q_a_layernorm): MiniCPMRMSNorm()
          (q_b_proj): Linear(in_features=768, out_features=3840, bias=False)
          (kv_a_proj_with_mqa): Linear(in_features=2560, out_features=288, bias=False)
          (kv_a_layernorm): MiniCPMRMSNorm()
          (kv_b_proj): Linear(in_features=256, out_features=5120, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): MiniCPMLongRoPE()
        )
        (mlp): MiniCPMMLP(
          (gate_proj): Linear(in_features=2560, out_features=6400, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6400, bias=False)
          (down_proj): Linear(in_features=6400, out_features=2560, bias=False)
   

# 输出模型元信息和张量信息

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载预训练模型和分词器
model_path = "/home/ztf/cpm"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
model.eval()

#  输出模型的元信息
print(model.config)
# with torch.no_grad():
#     for name, param in model.named_parameters():
#         print(f"Layer: {name} | Shape Type: {param.shape}")
# if name == "transformer.ln_f.weight":
#     print(f"Layer: {name} | Shape Type: {param.shape}| Data Type: {param}")
# if name == "transformer.ln_f.bias":
#     print(f"Layer: {name} | Shape Type: {param.shape} | Data Type: {param}")
#             df = pd.DataFrame(param.numpy())
#             # df.to_csv('output.csv', index=False)
#         # print(f"Layer: {name} | Shape: {param.shape}")
#     # for name, module in model.named_modules():
#     #      print(f"Name: {name}, Module: {module}")

MiniCPM3Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/home/ztf/cpm",
  "architectures": [
    "MiniCPM3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_minicpm.MiniCPM3Config",
    "AutoModel": "modeling_minicpm.MiniCPM3Model",
    "AutoModelForCausalLM": "modeling_minicpm.MiniCPM3ForCausalLM",
    "AutoModelForSeq2SeqLM": "modeling_minicpm.MiniCPM3ForCausalLM",
    "AutoModelForSequenceClassification": "modeling_minicpm.MiniCPM3ForSequenceClassification"
  },
  "bos_token_id": 1,
  "dim_model_base": 256,
  "eos_token_id": [
    2,
    73440
  ],
  "head_dim": 96,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.1,
  "intermediate_size": 6400,
  "kv_lora_rank": 256,
  "max_position_embeddings": 32768,
  "model_type": "minicpm3",
  "num_attention_heads": 40,
  "num_hidden_layers": 62,
  "num_key_value_heads": 40,
  "pretraining_tp": 1,
  "q_lora_rank": 768,
  "qk_nope_

# 输出每层打印信息

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def print_tensor_elements(tensor, label="Tensor", num_elements=5):
    if isinstance(tensor, torch.Tensor):
        elements = tensor.flatten()
        print(f"{label}: shape={tensor.shape}")
        print("First 5 elements:", elements[:num_elements].tolist())
        print(
            "Last 5 elements:",
            (
                elements[-num_elements:].tolist()
                if len(elements) >= num_elements
                else elements.tolist()
            ),
        )
    else:
        print(f"{label}: Not a Tensor")


def hook_fn(module, input, output):
    print(f"Layer: {module.__class__.__name__}")

    # 处理输入张量
    if input and isinstance(input[0], torch.Tensor):
        print_tensor_elements(input[0], label="Input")
    else:
        print("error")
    # 处理输出张量
    if input and isinstance(output[0], torch.Tensor):
        print_tensor_elements(output[0], label="Input")
    else:
        print("error")

    print("-" * 50)


# 选择设备
device_id = 0  # ← 修改这里选择GPU编号
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")
# 加载预训练模型和分词器
model_path = r"F:\edged\cpm"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
model.eval()
# 遍历所有子模块并注册钩子
hooks = []
for name, module in model.named_modules():
    if not isinstance(module, (torch.nn.ModuleList, torch.nn.Sequential)):
        hooks.append(module.register_forward_hook(hook_fn))
        # if name == 'transformer.ln_f':
        #     hooks.append(module.register_forward_hook(hook_fn))

# 手动构建张量并进行推理
inputs = "Once upon a time,"
generated_tokens = torch.tensor([[59422]])
# 将文本转换为模型输入
input_ids = tokenizer(inputs, return_tensors="pt").input_ids

# 使用模型进行推理
with torch.no_grad():  # 确保推理过程中不计算梯度以节省内存
    outputs = model.generate(generated_tokens, max_length=2, do_sample=True)
print(outputs)
for i in range(outputs.shape[0]):  # 遍历所有生成的序列
    print(tokenizer.decode(outputs[i], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Layer: Embedding
Input: shape=torch.Size([1, 1])
First 5 elements: [59422]
Last 5 elements: [59422]
Input: shape=torch.Size([1, 2560])
First 5 elements: [-0.030029296875, 0.06201171875, 0.08203125, -0.0262451171875, -0.0078125]
Last 5 elements: [-0.0576171875, -0.0050048828125, -0.0162353515625, 0.06494140625, -0.0595703125]
--------------------------------------------------
Layer: MiniCPMRMSNorm
Input: shape=torch.Size([1, 1, 2560])
First 5 elements: [-0.3603515625, 0.744140625, 0.984375, -0.31494140625, -0.09375]
Last 5 elements: [-0.69140625, -0.06005859375, -0.19482421875, 0.779296875, -0.71484375]
Input: shape=torch.Size([1, 2560])
First 5 elements: [-0.6367413401603699, 1.3148967027664185, 1.7393909692764282, -0.556501567363739, -0.16565628349781036]
Last 5 elements: [-1.2217150926589966, -0.10612355172634125, -0.3442544639110565, 1.377017855644226, -1.2631291151046753]
--------------------------------------------------
Layer: Linear
Input: shape=torch.Size([1, 1, 2560])
First 5 

# rope_ext

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype=torch.float32
max_seq_len_cached = 1
dim=32 #  qk_rope_head_dim
base=10000.0 #rope_theta

inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float().to(device) / dim))

long_factor=torch.tensor([
      1.0591234137867171,
      1.1241891283591912,
      1.2596935748670968,
      1.5380380402321725,
      2.093982484148734,
      3.1446935121267696,
      4.937952647693647,
      7.524541999994549,
      10.475458000005451,
      13.062047352306353,
      14.85530648787323,
      15.906017515851266,
      16.461961959767827,
      16.740306425132907,
      16.87581087164081,
      16.940876586213285
    ])
original_max_position_embeddings=32768
short_factor= torch.tensor([
      1.0591234137867171,
      1.1241891283591912,
      1.2596935748670968,
      1.5380380402321725,
      2.093982484148734,
      3.1446935121267696,
      4.937952647693647,
      7.524541999994549,
      10.475458000005451,
      13.062047352306353,
      14.85530648787323,
      15.906017515851266,
      16.461961959767827,
      16.740306425132907,
      16.87581087164081,
      16.940876586213285
    ])
       
t = torch.arange(max_seq_len_cached, device=device, dtype=dtype)

ext_factors = torch.tensor(short_factor, dtype=torch.float32, device=device)

freqs = torch.mul(
            torch.outer(t, 1.0 / ext_factors).to(device=device),
            inv_freq.to(device=device).to(dtype)
        )
emb = torch.cat((freqs, freqs), dim=-1)
print(emb.cos())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])


/tmp/ipykernel_1001639/3110317160.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ext_factors = torch.tensor(short_factor, dtype=torch.float32, device=device)
